In [1]:
import pandas as pd
from dotenv import load_dotenv
import os 
import csv
import psycopg2 
import json
load_dotenv()

URL_DB = os.environ.get("URL_DB")
CONN = psycopg2.connect(URL_DB)
CURSOR = CONN.cursor()

In [3]:
# NOTE: GROUP LOCATION BY COUNTRIES. 
# NOTE: WHERE IS LOCATION == NONE?
# NOTE: WHERE IS DESCRIPTION == NONE?


## TODO: NEED TO TRANSFROM COMMON TWO WORDS SUCH AS UNITED STATES INTO "USA"; "NEW YORK", 

## TODO: AS SOME LOCATIONS CONTAIN MORE THAN A WORD, IT SHOULD ADD THE NEXT WORD IF THE RESULT IS UNKNOWN, AND IF THEIR INDEX ARE THE SAME. ELSE FINALLY ASSING UNKNOWN AGAIN

timestamp = "2024-05-01 00:00:00.000000"

CURSOR.execute(
	f"SELECT location FROM main_jobs  WHERE timestamp > '{timestamp}'"
)
new_data = CURSOR.fetchall()

df = pd.DataFrame(new_data)


In [16]:
def load_json_file(file_path: str) -> dict:
	with open(file_path, 'r') as file:
		return json.load(file)

def save_json_file(data: dict, file_path: str) -> None:
	with open(file_path, 'w') as file:
		json.dump(data, file, indent=4)

In [8]:

# Get the column name (there's only one)
location = df.columns[0]

# Get unique values
#unique_values = df[location].str.strip().unique()

df[location] = df[location].str.strip().str.split()

df = df.explode(location)


In [9]:
df

,0
0,Remote
0,USA
0,Europe
0,(remote)
0,Remote
...,...
7993,Arab
7993,Emirates
7994,Worldwide
7995,Pakistan


In [22]:


## TODO: AS SOME LOCATIONS CONTAIN MORE THAN A WORD, IT SHOULD ADD THE NEXT WORD IF THE RESULT IS UNKNOWN, AND IF THEIR INDEX ARE THE SAME. ELSE FINALLY ASSING UNKNOWN AGAIN. MAKE A FUNCTION THAT CONCATENATES THE WORD THAT COULD NOT BE FOUND IN LOC.LOWER AND ADD ITS NEXT WORD ONLY IF THEIR INDEX ARE EQUAL. FOR EXAMPLE, THIS FUNCTION WOULD NOT MATCH THE WORD "SAN" BUT IF WE DO THIS, THEN THE CONCATENATED STRING WOULD BE "SAN FRANCISCO", WHICH SHOULD FIND ITS MATCH.

def add_location_tags(df: pd.DataFrame, json_file_path: str) -> pd.DataFrame:
	location_data = load_json_file(json_file_path)
	def find_location_tag(word: str) -> str:
		word_str = str(word)
		for continent, countries in location_data.items():
			for country in countries['Countries']:
				for country_name, locations in country.items():
					for loc in locations:
						if word_str.lower() in loc.lower():
							return country_name
						else:
							
					if word_str.lower() == country_name.lower():
						return country_name
			if word_str.lower() == continent.lower():
				return continent
		return "Unknown"
	
	df['location_tag'] = df[0].apply(find_location_tag)
	return df

json_file_path = '/root/JobsCrawler/src/notebooks/all_locations_transformed.json'
result_df = add_location_tags(df, json_file_path)
print(result_df)

result_df.to_csv("/root/JobsCrawler/src/notebooks/country_mapping.csv")

              0   location_tag
0        Remote        Unknown
0           USA  United States
0        Europe         Europe
0      (remote)        Unknown
0        Remote        Unknown
...         ...            ...
7993       Arab        Unknown
7993   Emirates        Unknown
7994  Worldwide       Anywhere
7995   Pakistan       Pakistan
7996   Pakistan       Pakistan

[38544 rows x 2 columns]


In [15]:
def transform_data(input_data: dict[str, dict[str, list[dict[str, str | list[dict[str, str]]]]]]) -> dict[str, dict[str, list[dict[str, list[str]]]]]:
	result = {}
	for continent, data in input_data.items():
		result[continent] = {"Countries": []}
		for country in data["Countries"]:
			country_name = country["country_name"]
			country_code = country["country_code"]
			capital_english = country["capital_english"]
			
			subdivisions = []
			if isinstance(country["subdivisions"], list):
				subdivisions = [sub["subdivisions_name"] for sub in country["subdivisions"]]
			elif country["subdivisions"] != "NaN":
				subdivisions = [country["subdivisions"]]
			
			transformed_country = {
				country_name: [country_code, capital_english] + subdivisions
			}
			result[continent]["Countries"].append(transformed_country)
	
	return result

#Example usage:

input_file = "/root/JobsCrawler/src/notebooks/all_locations.json"
output_file = "/root/JobsCrawler/src/notebooks/all_locations_transformed.json"

input_data = load_json_file(input_file)
transformed_data = transform_data(input_data)
save_json_file(transformed_data, output_file)

print("Data transformation complete. Result saved to", output_file)

Data transformation complete. Result saved to /root/JobsCrawler/src/notebooks/all_locations_transformed.json


In [ ]:
def clean(location_df):
	

In [15]:
import re

def get_country(location):
	# Remove 'Remote' and '(remote)' from the location string
	clean_location = re.sub(r'\bRemote\b|\(remote\)', '', location, flags=re.IGNORECASE).strip()
	
	for country, keywords in COUNTRY_MAPPING.items():
		for keyword in keywords:
			if keyword.lower() in clean_location.lower():
				return country
	
	
	return "Other"

grouped_locations = {}

for location in unique_values:
	country = get_country(location)
	if country not in grouped_locations:
		grouped_locations[country] = []
	grouped_locations[country].append(location)




In [16]:
with open("/root/JobsCrawler/src/notebooks/countries.json", mode="w+") as file:
	json.dump(grouped_locations, file, indent=4)